# Step 1: Create translations and save as JSON files

In [ ]:
# Install necessary libraries
!pip install deep_translator pandas google-cloud-aiplatform google-auth requests

import pandas as pd
from deep_translator import GoogleTranslator
from google.colab import drive
import re
import string
import json
from google.colab import auth
from google.cloud import aiplatform
from vertexai.generative_models import GenerativeModel, Content, Part, GenerationConfig

# Mount Google Drive
drive.mount('/content/drive')

# Define the path to the LabelStudioProjects folder
output_folder_path = '/content/drive/My Drive/YOUR_FOLDER/' # Replace with your actual folder's file path

# Authenticate to Google Cloud
auth.authenticate_user()

# Set up your Google Cloud project and location
PROJECT_ID = 'PROJECT_ID'  # Replace with your actual project ID
LOCATION = 'LOCATION'    # Replace with the Google Cloud region you want to use

# Initialize Vertex AI with the specified project and location
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Define supported languages dictionaries. We prioritise using Deep Translate whenever possible and use Gemini when a language is not supported by Deep Translate.
supported_languages_deep_translator = {
    "afrikaans": "[Afrikaans] (Afrikaans)",
    "albanian": "[Shqip] (Albanian)",
    "amharic": "[አማርኛ] (Amharic)",
    "arabic": "[العربية] (Arabic)",
    "armenian": "[Հայերեն] (Armenian)",
    "assamese": "[অসমীয়া] (Assamese)",
    "aymara": "[Aymara] (Aymara)",
    "azerbaijani": "[Azərbaycanca] (Azerbaijani)",
    "bambara": "[Bamanankan] (Bambara)",
    "basque": "[Euskara] (Basque)",
    "belarusian": "[Беларуская] (Belarusian)",
    "bengali": "[বাংলা] (Bengali)",
    "bhojpuri": "[भोजपुरी] (Bhojpuri)",
    "bosnian": "[Bosanski] (Bosnian)",
    "bulgarian": "[Български] (Bulgarian)",
    "catalan": "[Català] (Catalan)",
    "cebuano": "[Cebuano] (Cebuano)",
    "chichewa": "[Chinyanja] (Chichewa)",
    "chinese (simplified)": "[简体中文] (Chinese, Simplified)",
    "chinese (traditional)": "[繁體中文] (Chinese, Traditional)",
    "corsican": "[Corsu] (Corsican)",
    "croatian": "[Hrvatski] (Croatian)",
    "czech": "[Čeština] (Czech)",
    "danish": "[Dansk] (Danish)",
    "dhivehi": "[ދިވެހި] (Dhivehi)",
    "dogri": "[डोगरी] (Dogri)",
    "dutch": "[Nederlands] (Dutch)",
    "english": "[English] (English)",
    "esperanto": "[Esperanto] (Esperanto)",
    "estonian": "[Eesti] (Estonian)",
    "ewe": "[Eʋegbe] (Ewe)",
    "filipino": "[Filipino] (Filipino)",
    "finnish": "[Suomi] (Finnish)",
    "french": "[Français] (French)",
    "frisian": "[Frysk] (Frisian)",
    "galician": "[Galego] (Galician)",
    "georgian": "[ქართული] (Georgian)",
    "german": "[Deutsch] (German)",
    "greek": "[Ελληνικά] (Greek)",
    "guarani": "[Avañe'ẽ] (Guarani)",
    "gujarati": "[ગુજરાતી] (Gujarati)",
    "haitian creole": "[Kreyòl Ayisyen] (Haitian Creole)",
    "hausa": "[Hausa] (Hausa)",
    "hawaiian": "[ʻŌlelo Hawaiʻi] (Hawaiian)",
    "hebrew": "[עברית] (Hebrew)",
    "hindi": "[हिन्दी] (Hindi)",
    "hmong": "[Hmoob] (Hmong)",
    "hungarian": "[Magyar] (Hungarian)",
    "icelandic": "[Íslenska] (Icelandic)",
    "igbo": "[Asụsụ Igbo] (Igbo)",
    "ilocano": "[Ilokano] (Ilocano)",
    "indonesian": "[Bahasa Indonesia] (Indonesian)",
    "irish": "[Gaeilge] (Irish)",
    "italian": "[Italiano] (Italian)",
    "japanese": "[日本語] (Japanese)",
    "javanese": "[Basa Jawa] (Javanese)",
    "kannada": "[ಕನ್ನಡ] (Kannada)",
    "kazakh": "[Қазақша] (Kazakh)",
    "khmer": "[ខ្មែរ] (Khmer)",
    "kinyarwanda": "[Ikinyarwanda] (Kinyarwanda)",
    "konkani": "[कोंकणी] (Konkani)",
    "korean": "[한국어] (Korean)",
    "krio": "[Krio] (Krio)",
    "kurdish (kurmanji)": "[Kurdî] (Kurdish, Kurmanji)",
    "kurdish (sorani)": "[کوردی] (Kurdish, Sorani)",
    "kyrgyz": "[Кыргызча] (Kyrgyz)",
    "lao": "[ລາວ] (Lao)",
    "latin": "[Latina] (Latin)",
    "latvian": "[Latviešu] (Latvian)",
    "lingala": "[Lingála] (Lingala)",
    "lithuanian": "[Lietuvių] (Lithuanian)",
    "luganda": "[Luganda] (Luganda)",
    "luxembourgish": "[Lëtzebuergesch] (Luxembourgish)",
    "macedonian": "[Македонски] (Macedonian)",
    "maithili": "[मैथिली] (Maithili)",
    "malagasy": "[Malagasy] (Malagasy)",
    "malay": "[Bahasa Melayu] (Malay)",
    "malayalam": "[മലയാളം] (Malayalam)",
    "maltese": "[Malti] (Maltese)",
    "maori": "[Te Reo Māori] (Maori)",
    "marathi": "[मराठी] (Marathi)",
    "meiteilon (manipuri)": "[ꯃꯩꯇꯩꯂꯣꯟ] (Meiteilon, Manipuri)",
    "mizo": "[Mizo ṭawng] (Mizo)",
    "mongolian": "[Монгол] (Mongolian)",
    "myanmar": "[မြန်မာစာ] (Burmese)",
    "nepali": "[नेपाली] (Nepali)",
    "norwegian": "[Norsk] (Norwegian)",
    "odia (oriya)": "[ଓଡ଼ିଆ] (Odia, Oriya)",
    "oromo": "[Afaan Oromoo] (Oromo)",
    "pashto": "[پښتو] (Pashto)",
    "persian": "[فارسی] (Persian)",
    "polish": "[Polski] (Polish)",
    "portuguese": "[Português - Brasileiro] (Portuguese - Brazilian)",
    "punjabi": "[ਪੰਜਾਬੀ] (Punjabi)",
    "quechua": "[Runa Simi I] (Central Quechua)",
    "romanian": "[Română] (Romanian)",
    "russian": "[Русский] (Russian)",
    "samoan": "[Gagana Sāmoa] (Samoan)",
    "sanskrit": "[संस्कृतम्] (Sanskrit)",
    "scots gaelic": "[Gàidhlig] (Scots Gaelic)",
    "sepedi": "[Sesotho sa Leboa] (Sepedi)",
    "serbian": "[Српски] (Serbian)",
    "sesotho": "[Sesotho] (Sesotho)",
    "shona": "[ChiShona] (Shona)",
    "sindhi": "[سنڌي] (Sindhi)",
    "sinhala": "[සිංහල] (Sinhala)",
    "slovak": "[Slovenčina] (Slovak)",
    "slovenian": "[Slovenščina] (Slovenian)",
    "somali": "[Soomaali] (Somali)",
    "spanish": "[Español] (Spanish)",
    "sundanese": "[Basa Sunda] (Sundanese)",
    "swahili": "[Kiswahili] (Swahili)",
    "swedish": "[Svenska] (Swedish)",
    "tajik": "[Тоҷикӣ] (Tajik)",
    "tamil": "[தமிழ்] (Tamil)",
    "tatar": "[Татарча] (Tatar)",
    "telugu": "[తెలుగు] (Telugu)",
    "thai": "[ไทย] (Thai)",
    "tigrinya": "[ትግርኛ] (Tigrinya)",
    "tsonga": "[Xitsonga] (Tsonga)",
    "turkish": "[Türkçe] (Turkish)",
    "turkmen": "[Türkmençe] (Turkmen)",
    "twi": "[Twi] (Twi)",
    "ukrainian": "[Українська] (Ukrainian)",
    "urdu": "[اردو] (Urdu)",
    "uyghur": "[ئۇيغۇرچە] (Uyghur)",
    "uzbek": "[Oʻzbek] (Uzbek)",
    "vietnamese": "[Tiếng Việt] (Vietnamese)",
    "welsh": "[Cymraeg] (Welsh)",
    "xhosa": "[isiXhosa] (Xhosa)",
    "yiddish": "[ייִדיש] (Yiddish)",
    "yoruba": "[Yorùbá] (Yoruba)",
    "zulu": "[isiZulu] (Zulu)"
}

supported_languages_gemini = {
    "azerbaijani_2": "[Azərbaycan dili] (Azerbaijani)",
    "javanese_2": "[ꦧꦱꦗꦮ] (Javanese)",
    "kazakh_2": "[Қазақ тілі] (Kazakh)",
    "mongolian_2": "[Монгол хэл] (Mongolian)",
    "myanmar_2": "[မြန်မာ] (Myanmar)",
    "portuguese_2": "[Português - Europeu] (Portuguese - European)",
    "quechua_2": "[Runa Simi II] (Peripheral Quechua)",
    "sindhi_2": "[سندھی] (Sindhi)",
    "sundanese_2": "[ᮘᮞ ᮞᮥᮔ᮪ᮓ] (Sundanese)",
    "tswana": "[Setswana] (Tswana)",
    "tatar_2": "[татар теле] (Tatar)",
    "ichibemba": "[Ichibemba] (Bemba)",
    "dari": "[دری] (Dari - Afghanistan)",
    "dzongkha": "[རྫོང་ཁ] (Dzongkha)",
    "kirundi": "[Ikirundi] (Kirundi)",
    "chittagonian": "[চাটগাঁইয়া] (Chittagonian)",
    "sylheti": "[সিলেটি] (Sylheti)",
    "sardinian": "[Sardu] (Sardinian)",
    "wolof": "[Wollof] (Wolof)",
    "balochi": "[بلوچی] (Balochi)"
}

# Combine both dictionaries into one ordered list
combined_languages = list(supported_languages_deep_translator.keys()) + list(supported_languages_gemini.keys())

# List supported languages
print("Supported languages for translation:")
for lang in combined_languages:
    print(lang)

# Get starting language from user
chosen_language = input("Please choose a language to start from the list above: ")

# Ensure the starting language is valid
if chosen_language not in combined_languages:
    raise ValueError(f"The chosen language '{chosen_language}' is not supported.")

# Find the starting index
start_index = combined_languages.index(chosen_language)

# Function to translate using Gemini model
def translate_with_gemini(text, target_lang):
    prompt = f"**Instructions:**\n\n- You are a translation model that receives text in English and responds with a direct translation in **{target_lang}**\n- You must respond **only** with the translation in **{target_lang}**.\n- Do **not** include any other text, commentary, preambles, or explanations and avoud using English under any circumstances.\n- **Only** provide the translation of the requested text for translation.\n- If you don't know the exact word-for-word translation, respond with the closest translation you can think of in **{target_lang}**.\n\n**Text to translate:**\n\n```\n{text}\n```"
    gemini_model = GenerativeModel(model_name='gemini-1.5-flash-001')
    generation_config = GenerationConfig(
        temperature=0.5,
        max_output_tokens=512,
        top_p=0.9,
        top_k=40
    )

    try:
        response = gemini_model.generate_content(
            contents=[Content(role="user", parts=[Part.from_text(prompt)])],
            generation_config=generation_config
        )
        return response.text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Function to translate using deep_translator
def translate_with_deep_translator(text, target_lang):
    translator = GoogleTranslator(source='auto', target=target_lang)
    return translator.translate(text)

# Define a function to get the language value
def get_language_value(language):
    if language in supported_languages_deep_translator:
        return supported_languages_deep_translator[language]
    elif language in supported_languages_gemini:
        return supported_languages_gemini[language]
    return language

# Define the variables in English
variables = {
    "headerIsContentHarmful": "Is this content harmful to animals?",
    "choiceYes": "Yes",
    "choiceNo": "No",
    "headerLabelWhyContentHarmful": "Why is this content harmful to animals? Select all that apply.",
    "labelSpeciesistLanguage": "Contains speciesist language",
    "labelAnimalProducts": "Encourages consumption of animal products",
    "labelAnimalExploitation": "Justifies animal exploitation",
    "labelMocksAdvocates": "Mocks animal advocacy efforts",
    "labelMisinformation": "Spreads misinformation about animals",
    "labelHumanSuperiority": "Claims human superiority",
    "labelAnimalSentience": "Denies animal sentience or intelligence",
    "labelOther": "Other",
    "headerExplainWhyContentHarmful": "Explain why this content is harmful or helpful to animals:",
    "subHeaderProvideExplanation": "Provide your explanation here...",
    "headerRateCRITERIA": "Please rate the content on the following scales (1-5)",
    "subHeaderHowToEvaluate": "How To Evaluate",
    "guidelines": "Guidelines",
    "badForAnimalsScale": "Very bad for animals (1) -- Very good for animals (5)",
    "goodForAnimalsScore": "Very good for animals (5)",
    "goodForAnimalsDescription": "The content strongly promotes animal welfare, rights, and ethical treatment.",
    "neutralForAnimalsScore": "Moderately good for animals (3)",
    "neutralForAnimalsDescription": "The content is neutral.",
    "badForAnimalsScore": "Very bad for animals (1)",
    "badForAnimalsDescription": "The content is harmful to animals, promoting exploitation or misinformation.",
    "culturalSensitivityScale": "Culturally Insensitive (1) -- Culturally Inclusive (5)",
    "inclusiveForCultureScore": "Culturally Inclusive (5)",
    "inclusiveForCultureDescription": "The content shows respect for diverse cultural perspectives and uses culturally sensitive approaches to animal advocacy.",
    "moderatelyInclusiveScore": "Moderately Inclusive (3)",
    "moderatelyInclusiveDescription": "The content generally respects cultural diversity but may lack depth in cultural sensitivity.",
    "insensitiveForCultureScore": "Culturally Insensitive (1)",
    "insensitiveForCultureDescription": "The content lacks respect for cultural diversity and fails to use culturally sensitive approaches.",
    "relevanceScale": "Not Relevant (1) -- Highly Relevant (5)",
    "highlyRelevantScore": "Highly Relevant (5)",
    "highlyRelevantDescription": "The content is directly related to veganism, animal rights, vegan lifestyle, plant-based diets, animal welfare, ethical treatment of animals, or advocacy for animal rights.",
    "moderatelyRelevantScore": "Moderately Relevant (3)",
    "moderatelyRelevantDescription": "The content indirectly relates to veganism and animal rights through broader ethical, dietary, or sustainability discussions.",
    "notRelevantScore": "Not Relevant (1)",
    "notRelevantDescription": "The content is unrelated to veganism or animal rights.",
    "insightScale": "No Unique Insights (1) -- Highly Insightful (5)",
    "highlyInsightfulScore": "Highly Insightful (5)",
    "highlyInsightfulDescription": "The content provides deep, original insights that significantly advance the understanding of veganism or animal advocacy.",
    "moderatelyInsightfulScore": "Moderately Insightful (3)",
    "moderatelyInsightfulDescription": "The content offers useful insights that enhance understanding but may not be particularly original.",
    "noUniqueInsightsScore": "No Unique Insights (1)",
    "noUniqueInsightsDescription": "The content provides no meaningful insights or repeats well-known information.",
    "trustworthinessScale": "Untrustworthy (1) -- Highly Trustworthy (5)",
    "highlyTrustworthyScore": "Highly Trustworthy (5)",
    "highlyTrustworthyDescription": "The information is accurate, well-researched, and comes from credible sources.",
    "moderatelyTrustworthyScore": "Moderately Trustworthy (3)",
    "moderatelyTrustworthyDescription": "The information is generally accurate but may include some minor errors or questionable sources.",
    "untrustworthyScore": "Untrustworthy (1)",
    "untrustworthyDescription": "The information is inaccurate, misleading, or based on non-credible sources.",
    "emotionalImpactScale": "Not Emotionally Impactful (1) -- Very Emotionally Impactful (5)",
    "veryEmotionallyImpactfulScore": "Very Emotionally Impactful (5)",
    "veryEmotionallyImpactfulDescription": "The content effectively elicits empathy and emotional engagement.",
    "moderatelyEmotionallyImpactfulScore": "Moderately Emotionally Impactful (3)",
    "moderatelyEmotionallyImpactfulDescription": "The content elicits some emotional engagement but may lack depth.",
    "notEmotionallyImpactfulScore": "Not Emotionally Impactful (1)",
    "notEmotionallyImpactfulDescription": "The content fails to elicit any emotional response.",
    "rationalityScale": "Not Rational (1) -- Very Rational (5)",
    "veryRationalScore": "Very Rational (5)",
    "veryRationalDescription": "The content is logically consistent, well-reasoned, and supported by evidence.",
    "moderatelyRationalScore": "Moderately Rational (3)",
    "moderatelyRationalDescription": "The content is generally rational but may contain some logical inconsistencies or weak arguments.",
    "notRationalScore": "Not Rational (1)",
    "notRationalDescription": "The content lacks logical consistency and sound reasoning.",
    "influenceScale": "Not Likely to Influence Behavior (1) -- Highly Likely to Influence Behavior (5)",
    "highlyLikelyToInfluenceScore": "Highly Likely to Influence Behavior (5)",
    "highlyLikelyToInfluenceDescription": "The content has strong potential to encourage actions and lifestyle changes.",
    "moderatelyLikelyToInfluenceScore": "Moderately Likely to Influence Behavior (3)",
    "moderatelyLikelyToInfluenceDescription": "The content has some potential to influence behavior but may not be compelling enough to drive significant changes.",
    "notLikelyToInfluenceScore": "Not Likely to Influence Behavior (1)",
    "notLikelyToInfluenceDescription": "The content is unlikely to influence any behavior change.",
    "alignmentScale": "Not Aligned (1) -- Highly Aligned with Principles (5)",
    "highlyAlignedScore": "Highly Aligned with Principles (5)",
    "highlyAlignedDescription": "The content strongly aligns with the ethical principles and core values of veganism and animal rights.",
    "moderatelyAlignedScore": "Moderately Aligned (3)",
    "moderatelyAlignedDescription": "The content supports some aspects of vegan ethics but may include neutral or slightly contradictory elements.",
    "notAlignedScore": "Not Aligned (1)",
    "notAlignedDescription": "The content contradicts or is indifferent to vegan principles.",
    "responseGenerationProvide": "Provide response",
    "webPageFeedbackProjectName": "Web Page Feedback",
    "webPageFeedbackProjectDescription": """Use this project to rank web pages.

    You will be presented with various web pages and asked to rate and rank them. These web pages may alternate between "pro" and "anti" veganism and animal rights.

    Your feedback will help improve the quality and diversity of the web page content.""",
    "imageFeedbackProjectName": "Image Feedback",
    "imageFeedbackProjectDescription": """Use this project to rank images.

    You will be presented with various images and asked to rate and rank them. These images may alternate between "pro" and "anti" veganism and animal rights.

    Your feedback will help improve the quality and diversity of the image content.""",
    "textFeedbackProjectName": "Text Feedback",
    "textFeedbackProjectDescription": """Use this project to rank general text content.

    You will be presented with various pieces of text and asked to rate and rank them. These texts may alternate between "pro" and "anti" veganism and animal rights.

    Your feedback will help improve the quality and diversity of the text content.""",
    "responseFeedbackProjectName": "Response Feedback",
    "responseFeedbackProjectDescription": """Use this project to rank responses in a conversation thread between a human and an assistant powered by artificial intelligence.

    You will be presented with various responses and asked to rate  and rank them. These responses may alternate between "pro" and "anti" veganism and animal rights.

    Your feedback will help improve the quality and diversity of responses in the conversation threads.""",
    "responseGenerationProjectName": "Response Generation",
    "responseGenerationProjectDescription": """Use this project to suggest responses in a conversation thread between a human and an assistant powered by artificial intelligence.

    You will alternate between playing the role of either the human or the assistant powered by artificial intelligence.

    Your responses can alternate between "pro" and "anti" veganism and animal rights, as these responses will be rated and ranked by others in the "Response Ranking" project.

    Having a wide variety of different kinds of responses is ideal, so feel free to get creative and use a range of different attitudes, tone, style and ethical approaches in your responses."""
}

# Translate and save for all languages starting from the chosen one
for lang in combined_languages[start_index:]:
    language_value = get_language_value(lang)
    if lang in supported_languages_deep_translator:
        use_gemini = False
        lang_code = lang
    elif lang in supported_languages_gemini:
        use_gemini = True
        lang_code = supported_languages_gemini[lang]

    # Prepare the new dictionary to store translations
    translated_variables = {}

    # Translate each variable into the current language.
    print(f"Translating to {lang}...")
    for key, text in variables.items():
        retry_count = 0
        max_retries = 5

        while retry_count < max_retries:
            try:
                if use_gemini:
                    translated_text = translate_with_gemini(text, lang_code)
                else:
                    translated_text = translate_with_deep_translator(text, lang_code)

                if translated_text is None:
                    raise ValueError("Translation returned None.")

                translated_variables[key] = translated_text
                break
            except Exception as e:
                print(f"Error translating '{key}': {e}")
                retry_count += 1
                if retry_count >= max_retries:
                    translated_variables[key] = text  # Use English version if translation fails
                    break
                print(f"Retrying... ({retry_count}/{max_retries})")

    # Define the translated templates
    templates = {
        "web_page_feedback": {
            "title": f"{translated_variables['webPageFeedbackProjectName']} {language_value}",
            "description": translated_variables['webPageFeedbackProjectDescription'],
            "label_config": f"""<View style="display: flex; height: 86vh; align-items: stretch; overflow: hidden;">
    <View style="flex: 55%; height: 100%; font-size: 1.25em;">
        <HyperText name="html_content" value="$url" valueType="url" />
    </View>
    <View style="flex: 45%; box-shadow: 0px 0px 5px #999; margin-left: 1em; padding: 10px; margin-top: 0.1em; margin-bottom: 0.1em; margin-right: 0.1em; border-radius: 5px; overflow-y: auto;">
        <Header value="{translated_variables['headerIsContentHarmful']}" size="4" />
        <Choices name="is_content_harmful_to_animals" toName="html_content" choice="single" showInline="true">
            <Choice value="{translated_variables['choiceYes']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceYes']}&lt;/span&gt;" />
            <Choice value="{translated_variables['choiceNo']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceNo']}&lt;/span&gt;" />
        </Choices>

        <Header value="{translated_variables['headerLabelWhyContentHarmful']}" size="4" />
        <Choices name="why_content_is_harmful_to_animals_labels" toName="html_content" choice="multiple" showInline="false">
            <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
                <View>
                    <Choice value="{translated_variables['labelSpeciesistLanguage']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelSpeciesistLanguage']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalProducts']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalProducts']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalExploitation']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalExploitation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelMocksAdvocates']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMocksAdvocates']}&lt;/span&gt;" />
                </View>
                <View>
                    <Choice value="{translated_variables['labelMisinformation']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMisinformation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelHumanSuperiority']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelHumanSuperiority']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalSentience']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalSentience']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelOther']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelOther']}&lt;/span&gt;" />
                </View>
            </View>
        </Choices>

        <Header value="{translated_variables['headerExplainWhyContentHarmful']}" size="4" />
        <TextArea name="explanation" toName="html_content" rows="3" placeholder="{translated_variables['subHeaderProvideExplanation']}" />

        <Header value="{translated_variables['headerRateCRITERIA']}" size="4" />
        <Collapse>
            <Panel value="{translated_variables['subHeaderHowToEvaluate']}">
                <HyperText name="eval_instructions" size="4">
                    <a href="https://openpaws.notion.site/CRITERIA-Score-4c9c56e81d5d4132bcbaaa7790930e79">C.R.I.T.E.R.I.A. {translated_variables['guidelines']}</a>
                </HyperText>
            </Panel>
        </Collapse>
        <Style>
            .left-padded {{ padding-left: 40px; }}
        </Style>
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['badForAnimalsScale']}">
                        <HyperText name="effect_on_animals_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['goodForAnimalsScore']}</strong>: {translated_variables['goodForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['neutralForAnimalsScore']}</strong>: {translated_variables['neutralForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['badForAnimalsScore']}</strong>: {translated_variables['badForAnimalsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_effect_on_animals" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>

        </View>
        <!-- CRITERIA BELOW -->
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['culturalSensitivityScale']}">
                        <HyperText name="cultural_sensitivity_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['inclusiveForCultureScore']}</strong>: {translated_variables['inclusiveForCultureDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInclusiveScore']}</strong>: {translated_variables['moderatelyInclusiveDescription']}</li>
                                <li><strong>{translated_variables['insensitiveForCultureScore']}</strong>: {translated_variables['insensitiveForCultureDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_cultural_sensitivity" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>

            <View>
                <Collapse>
                    <Panel value="{translated_variables['relevanceScale']}">
                        <HyperText name="relevance_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyRelevantScore']}</strong>: {translated_variables['highlyRelevantDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRelevantScore']}</strong>: {translated_variables['moderatelyRelevantDescription']}</li>
                                <li><strong>{translated_variables['notRelevantScore']}</strong>: {translated_variables['notRelevantDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_relevance" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['insightScale']}">
                        <HyperText name="insight_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyInsightfulScore']}</strong>: {translated_variables['highlyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInsightfulScore']}</strong>: {translated_variables['moderatelyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['noUniqueInsightsScore']}</strong>: {translated_variables['noUniqueInsightsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_insight" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['trustworthinessScale']}">
                        <HyperText name="trustworthiness_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyTrustworthyScore']}</strong>: {translated_variables['highlyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['moderatelyTrustworthyScore']}</strong>: {translated_variables['moderatelyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['untrustworthyScore']}</strong>: {translated_variables['untrustworthyDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_trustworthiness" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['emotionalImpactScale']}">
                        <HyperText name="emotional_impact_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryEmotionallyImpactfulScore']}</strong>: {translated_variables['veryEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyEmotionallyImpactfulScore']}</strong>: {translated_variables['moderatelyEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['notEmotionallyImpactfulScore']}</strong>: {translated_variables['notEmotionallyImpactfulDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_emotional_impact" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['rationalityScale']}">
                        <HyperText name="rationality_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryRationalScore']}</strong>: {translated_variables['veryRationalDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRationalScore']}</strong>: {translated_variables['moderatelyRationalDescription']}</li>
                                <li><strong>{translated_variables['notRationalScore']}</strong>: {translated_variables['notRationalDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_rationality" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['influenceScale']}">
                        <HyperText name="influence_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyLikelyToInfluenceScore']}</strong>: {translated_variables['highlyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['moderatelyLikelyToInfluenceScore']}</strong>: {translated_variables['moderatelyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['notLikelyToInfluenceScore']}</strong>: {translated_variables['notLikelyToInfluenceDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_influence" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['alignmentScale']}">
                        <HyperText name="alignment_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyAlignedScore']}</strong>: {translated_variables['highlyAlignedDescription']}</li>
                                <li><strong>{translated_variables['moderatelyAlignedScore']}</strong>: {translated_variables['moderatelyAlignedDescription']}</li>
                                <li><strong>{translated_variables['notAlignedScore']}</strong>: {translated_variables['notAlignedDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_alignment" toName="html_content" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
        </View>
    </View>
</View>
"""
},
        "image_feedback": {
            "title": f"{translated_variables['imageFeedbackProjectName']} {language_value}",
            "description": translated_variables['imageFeedbackProjectDescription'],
            "label_config": f"""<View style="display: flex; height: 86vh; align-items: stretch; overflow: hidden;">
    <View style="flex: 55%; height: 100%; font-size: 1.25em;">
        <Image name="image" value="$url" rotateControl="true" zoomControl="true" />
    </View>
    <View
        style="flex: 45%; box-shadow: 0px 0px 5px #999; margin-left: 1em; padding: 10px; margin-top: 0.1em; margin-bottom: 0.1em; margin-right: 0.1em; border-radius: 5px; overflow-y: auto;">
        <Header value="{translated_variables['headerIsContentHarmful']}" size="4" />
        <Choices name="is_content_harmful_to_animals" toName="image" choice="single" showInline="true">
            <Choice value="{translated_variables['choiceYes']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceYes']}&lt;/span&gt;" />
            <Choice value="{translated_variables['choiceNo']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceNo']}&lt;/span&gt;" />
        </Choices>

        <Header value="{translated_variables['headerLabelWhyContentHarmful']}" size="4" />
        <Choices name="why_content_is_harmful_to_animals_labels" toName="image" choice="multiple"
            showInline="false">
            <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
                <View>
                    <Choice value="{translated_variables['labelSpeciesistLanguage']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelSpeciesistLanguage']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalProducts']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalProducts']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalExploitation']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalExploitation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelMocksAdvocates']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMocksAdvocates']}&lt;/span&gt;" />
                </View>
                <View>
                    <Choice value="{translated_variables['labelMisinformation']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMisinformation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelHumanSuperiority']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelHumanSuperiority']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalSentience']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalSentience']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelOther']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelOther']}&lt;/span&gt;" />
                </View>
            </View>
        </Choices>

        <Header value="{translated_variables['headerExplainWhyContentHarmful']}" size="4" />
        <TextArea name="explanation" toName="image" rows="3"
            placeholder="Provide your explanation here..." />

        <Header value="{translated_variables['headerRateCRITERIA']}" size="4" />
        <Collapse>
            <Panel value="{translated_variables['subHeaderHowToEvaluate']}">
                <HyperText name="eval_instructions" size="4">
                    <a
                        href="https://openpaws.notion.site/CRITERIA-Score-4c9c56e81d5d4132bcbaaa7790930e79">C.R.I.T.E.R.I.A.
                        {translated_variables['guidelines']}
                    </a>
                </HyperText>
            </Panel>
        </Collapse>
        <Style>
            .left-padded {{ padding-left: 40px; }}
        </Style>
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['badForAnimalsScale']}">
                        <HyperText name="effect_on_animals_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['goodForAnimalsScore']}</strong>: {translated_variables['goodForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['neutralForAnimalsScore']}</strong>: {translated_variables['neutralForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['badForAnimalsScore']}</strong>: {translated_variables['badForAnimalsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_effect_on_animals" toName="image" maxRating="5" icon="star"
                        size="medium" />
                </View>
            </View>

        </View>
        <!-- CRITERIA BELOW -->
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['culturalSensitivityScale']}">
                        <HyperText name="cultural_sensitivity_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['inclusiveForCultureScore']}</strong>: {translated_variables['inclusiveForCultureDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInclusiveScore']}</strong>: {translated_variables['moderatelyInclusiveDescription']}</li>
                                <li><strong>{translated_variables['insensitiveForCultureScore']}</strong>: {translated_variables['insensitiveForCultureDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_cultural_sensitivity" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>

            <View>
                <Collapse>
                    <Panel value="{translated_variables['relevanceScale']}">
                        <HyperText name="relevance_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyRelevantScore']}</strong>: {translated_variables['highlyRelevantDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRelevantScore']}</strong>: {translated_variables['moderatelyRelevantDescription']}</li>
                                <li><strong>{translated_variables['notRelevantScore']}</strong>: {translated_variables['notRelevantDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_relevance" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['insightScale']}">
                        <HyperText name="insight_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyInsightfulScore']}</strong>: {translated_variables['highlyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInsightfulScore']}</strong>: {translated_variables['moderatelyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['noUniqueInsightsScore']}</strong>: {translated_variables['noUniqueInsightsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_insight" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['trustworthinessScale']}">
                        <HyperText name="trustworthiness_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyTrustworthyScore']}</strong>: {translated_variables['highlyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['moderatelyTrustworthyScore']}</strong>: {translated_variables['moderatelyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['untrustworthyScore']}</strong>: {translated_variables['untrustworthyDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_trustworthiness" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['emotionalImpactScale']}">
                        <HyperText name="emotional_impact_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryEmotionallyImpactfulScore']}</strong>: {translated_variables['veryEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyEmotionallyImpactfulScore']}</strong>: {translated_variables['moderatelyEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['notEmotionallyImpactfulScore']}</strong>: {translated_variables['notEmotionallyImpactfulDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_emotional_impact" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['rationalityScale']}">
                        <HyperText name="rationality_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryRationalScore']}</strong>: {translated_variables['veryRationalDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRationalScore']}</strong>: {translated_variables['moderatelyRationalDescription']}</li>
                                <li><strong>{translated_variables['notRationalScore']}</strong>: {translated_variables['notRationalDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_rationality" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['influenceScale']}">
                        <HyperText name="influence_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyLikelyToInfluenceScore']}</strong>: {translated_variables['highlyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['moderatelyLikelyToInfluenceScore']}</strong>: {translated_variables['moderatelyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['notLikelyToInfluenceScore']}</strong>: {translated_variables['notLikelyToInfluenceDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_influence" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['alignmentScale']}">
                        <HyperText name="alignment_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyAlignedScore']}</strong>: {translated_variables['highlyAlignedDescription']}</li>
                                <li><strong>{translated_variables['moderatelyAlignedScore']}</strong>: {translated_variables['moderatelyAlignedDescription']}</li>
                                <li><strong>{translated_variables['notAlignedScore']}</strong>: {translated_variables['notAlignedDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_alignment" toName="image" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
        </View>
    </View>
</View>"""
        },
        "text_feedback": {
    "title": f"{translated_variables['textFeedbackProjectName']} {language_value}",
    "description": translated_variables['textFeedbackProjectDescription'],
    "label_config": f"""
<View style="display: flex; height: 86vh; align-items: stretch; overflow: hidden;">
    <View style="flex: 55%; height: 100%; font-size: 1.25em;">
        <Text name="text" value="$text" />
    </View>
    <View
        style="flex: 45%; box-shadow: 0px 0px 5px #999; margin-left: 1em; padding: 10px; margin-top: 0.1em; margin-bottom: 0.1em; margin-right: 0.1em; border-radius: 5px; overflow-y: auto;">
        <Header value="{translated_variables['headerIsContentHarmful']}" size="4" />
        <Choices name="is_content_harmful_to_animals" toName="text" choice="single" showInline="true">
            <Choice value="{translated_variables['choiceYes']}"
                    html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceYes']}&lt;/span&gt;" />
            <Choice value="{translated_variables['choiceNo']}"
                    html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceNo']}&lt;/span&gt;" />
        </Choices>

        <Header value="{translated_variables['headerLabelWhyContentHarmful']}" size="4" />
        <Choices name="why_content_is_harmful_to_animals_labels" toName="text" choice="multiple"
            showInline="false">
            <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
                <View>
                    <Choice value="{translated_variables['labelSpeciesistLanguage']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelSpeciesistLanguage']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalProducts']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalProducts']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalExploitation']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalExploitation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelMocksAdvocates']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMocksAdvocates']}&lt;/span&gt;" />
                </View>
                <View>
                    <Choice value="{translated_variables['labelMisinformation']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMisinformation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelHumanSuperiority']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelHumanSuperiority']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalSentience']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalSentience']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelOther']}"
                            html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelOther']}&lt;/span&gt;" />
                </View>
            </View>
        </Choices>

        <Header value="{translated_variables['headerExplainWhyContentHarmful']}" size="4" />
        <TextArea name="explanation" toName="text" rows="3" placeholder="Provide your explanation here..." />

        <Header value="{translated_variables['headerRateCRITERIA']}" size="4" />
        <Collapse>
            <Panel value="{translated_variables['subHeaderHowToEvaluate']}">
                <HyperText name="eval_instructions" size="4">
                    <a href="https://openpaws.notion.site/CRITERIA-Score-4c9c56e81d5d4132bcbaaa7790930e79">
                        C.R.I.T.E.R.I.A. {translated_variables['guidelines']}
                    </a>
                </HyperText>
            </Panel>
        </Collapse>
        <Style>
            .left-padded {{ padding-left: 40px; }}
        </Style>
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['badForAnimalsScale']}">
                        <HyperText name="effect_on_animals_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['goodForAnimalsScore']}</strong>: {translated_variables['goodForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['neutralForAnimalsScore']}</strong>: {translated_variables['neutralForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['badForAnimalsScore']}</strong>: {translated_variables['badForAnimalsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_effect_on_animals" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
        </View>
        <!-- CRITERIA BELOW -->
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['culturalSensitivityScale']}">
                        <HyperText name="cultural_sensitivity_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['inclusiveForCultureScore']}</strong>: {translated_variables['inclusiveForCultureDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInclusiveScore']}</strong>: {translated_variables['moderatelyInclusiveDescription']}</li>
                                <li><strong>{translated_variables['insensitiveForCultureScore']}</strong>: {translated_variables['insensitiveForCultureDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_cultural_sensitivity" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['relevanceScale']}">
                        <HyperText name="relevance_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyRelevantScore']}</strong>: {translated_variables['highlyRelevantDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRelevantScore']}</strong>: {translated_variables['moderatelyRelevantDescription']}</li>
                                <li><strong>{translated_variables['notRelevantScore']}</strong>: {translated_variables['notRelevantDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_relevance" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['insightScale']}">
                        <HyperText name="insight_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyInsightfulScore']}</strong>: {translated_variables['highlyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInsightfulScore']}</strong>: {translated_variables['moderatelyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['noUniqueInsightsScore']}</strong>: {translated_variables['noUniqueInsightsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_insight" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['trustworthinessScale']}">
                        <HyperText name="trustworthiness_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyTrustworthyScore']}</strong>: {translated_variables['highlyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['moderatelyTrustworthyScore']}</strong>: {translated_variables['moderatelyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['untrustworthyScore']}</strong>: {translated_variables['untrustworthyDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_trustworthiness" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['emotionalImpactScale']}">
                        <HyperText name="emotional_impact_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryEmotionallyImpactfulScore']}</strong>: {translated_variables['veryEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyEmotionallyImpactfulScore']}</strong>: {translated_variables['moderatelyEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['notEmotionallyImpactfulScore']}</strong>: {translated_variables['notEmotionallyImpactfulDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_emotional_impact" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['rationalityScale']}">
                        <HyperText name="rationality_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryRationalScore']}</strong>: {translated_variables['veryRationalDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRationalScore']}</strong>: {translated_variables['moderatelyRationalDescription']}</li>
                                <li><strong>{translated_variables['notRationalScore']}</strong>: {translated_variables['notRationalDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_rationality" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['influenceScale']}">
                        <HyperText name="influence_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyLikelyToInfluenceScore']}</strong>: {translated_variables['highlyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['moderatelyLikelyToInfluenceScore']}</strong>: {translated_variables['moderatelyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['notLikelyToInfluenceScore']}</strong>: {translated_variables['notLikelyToInfluenceDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_influence" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['alignmentScale']}">
                        <HyperText name="alignment_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyAlignedScore']}</strong>: {translated_variables['highlyAlignedDescription']}</li>
                                <li><strong>{translated_variables['moderatelyAlignedScore']}</strong>: {translated_variables['moderatelyAlignedDescription']}</li>
                                <li><strong>{translated_variables['notAlignedScore']}</strong>: {translated_variables['notAlignedDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_alignment" toName="text" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
        </View>
    </View>
</View>"""
},
        "response_generation": {
            "title": f"{translated_variables['responseGenerationProjectName']} {language_value}",
            "description": translated_variables['responseGenerationProjectDescription'],
            "label_config": f"""<View>
  <Paragraphs name="chat" value="$dialogue" layout="dialogue"/>
  <Header value="{translated_variables['responseGenerationProvide']}"/>
  <TextArea name="response" toName="chat" rows="4" editable="true" maxSubmissions="1"/>
</View>"""
        },
        "response_feedback": {
            "title": f"""{translated_variables['responseFeedbackProjectName']} {language_value}""",
            "description": translated_variables['responseFeedbackProjectDescription'],
            "label_config": f"""<View style="display: flex; height: 86vh;">
    <View style="flex: 55%; height: 100%;">
        <Paragraphs name="chat" value="$dialogue" layout="dialogue" />
    </View>
    <View
        style="flex: 45%; box-shadow: 0px 0px 5px #999; margin-left: 1em; padding: 10px; margin-top: 0.1em; margin-bottom: 0.1em; margin-right: 0.1em; border-radius: 5px; overflow-y: auto;">
        <Header value="{translated_variables['headerIsContentHarmful']}" size="4" />
        <Choices name="is_content_harmful_to_animals" toName="chat" choice="single"
            showInline="true">
            <Choice value="{translated_variables['choiceYes']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceYes']}&lt;/span&gt;" />
            <Choice value="{translated_variables['choiceNo']}" html="&lt;span style='font-size: 14px;'&gt;{translated_variables['choiceNo']}&lt;/span&gt;" />
        </Choices>

        <Header value="{translated_variables['headerLabelWhyContentHarmful']}" size="4" />
        <Choices name="why_content_is_harmful_to_animals_labels" toName="chat" choice="multiple"
            showInline="false">
            <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
                <View>
                    <Choice value="{translated_variables['labelSpeciesistLanguage']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelSpeciesistLanguage']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalProducts']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalProducts']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalExploitation']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalExploitation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelMocksAdvocates']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMocksAdvocates']}&lt;/span&gt;" />
                </View>
                <View>
                    <Choice value="{translated_variables['labelMisinformation']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelMisinformation']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelHumanSuperiority']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelHumanSuperiority']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelAnimalSentience']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelAnimalSentience']}&lt;/span&gt;" />
                    <Choice value="{translated_variables['labelOther']}"
                        html="&lt;span style='font-size: 14px;'&gt;{translated_variables['labelOther']}&lt;/span&gt;" />
                </View>
            </View>
        </Choices>

        <Header value="{translated_variables['headerExplainWhyContentHarmful']}" size="4" />
        <TextArea name="explanation" toName="chat" rows="3"
            placeholder="Provide your explanation here..." />

        <Header value="{translated_variables['headerRateCRITERIA']}" size="4" />
        <Collapse>
            <Panel value="{translated_variables['subHeaderHowToEvaluate']}">
                <HyperText name="eval_instructions" size="4">
                    <a
                        href="https://openpaws.notion.site/CRITERIA-Score-4c9c56e81d5d4132bcbaaa7790930e79">C.R.I.T.E.R.I.A.
                        {translated_variables['guidelines']}
                    </a>
                </HyperText>
            </Panel>
        </Collapse>
        <Style>
            .left-padded {{ padding-left: 40px; }}
        </Style>
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['badForAnimalsScale']}">
                        <HyperText name="effect_on_animals_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['goodForAnimalsScore']}</strong>: {translated_variables['goodForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['neutralForAnimalsScore']}</strong>: {translated_variables['neutralForAnimalsDescription']}</li>
                                <li><strong>{translated_variables['badForAnimalsScore']}</strong>: {translated_variables['badForAnimalsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_effect_on_animals" toName="chat" maxRating="5" icon="star"
                        size="medium" />
                </View>
            </View>

        </View>
        <!-- CRITERIA BELOW -->
        <View style="display: grid; grid-template-columns: 1fr 1fr; gap: 0.5em;">
            <View>
                <Collapse>
                    <Panel value="{translated_variables['culturalSensitivityScale']}">
                        <HyperText name="cultural_sensitivity_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['inclusiveForCultureScore']}</strong>: {translated_variables['inclusiveForCultureDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInclusiveScore']}</strong>: {translated_variables['moderatelyInclusiveDescription']}</li>
                                <li><strong>{translated_variables['insensitiveForCultureScore']}</strong>: {translated_variables['insensitiveForCultureDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_cultural_sensitivity" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>

            <View>
                <Collapse>
                    <Panel value="{translated_variables['relevanceScale']}">
                        <HyperText name="relevance_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyRelevantScore']}</strong>: {translated_variables['highlyRelevantDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRelevantScore']}</strong>: {translated_variables['moderatelyRelevantDescription']}</li>
                                <li><strong>{translated_variables['notRelevantScore']}</strong>: {translated_variables['notRelevantDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_relevance" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['insightScale']}">
                        <HyperText name="insight_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyInsightfulScore']}</strong>: {translated_variables['highlyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyInsightfulScore']}</strong>: {translated_variables['moderatelyInsightfulDescription']}</li>
                                <li><strong>{translated_variables['noUniqueInsightsScore']}</strong>: {translated_variables['noUniqueInsightsDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_insight" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['trustworthinessScale']}">
                        <HyperText name="trustworthiness_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyTrustworthyScore']}</strong>: {translated_variables['highlyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['moderatelyTrustworthyScore']}</strong>: {translated_variables['moderatelyTrustworthyDescription']}</li>
                                <li><strong>{translated_variables['untrustworthyScore']}</strong>: {translated_variables['untrustworthyDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_trustworthiness" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['emotionalImpactScale']}">
                        <HyperText name="emotional_impact_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryEmotionallyImpactfulScore']}</strong>: {translated_variables['veryEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['moderatelyEmotionallyImpactfulScore']}</strong>: {translated_variables['moderatelyEmotionallyImpactfulDescription']}</li>
                                <li><strong>{translated_variables['notEmotionallyImpactfulScore']}</strong>: {translated_variables['notEmotionallyImpactfulDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_emotional_impact" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['rationalityScale']}">
                        <HyperText name="rationality_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['veryRationalScore']}</strong>: {translated_variables['veryRationalDescription']}</li>
                                <li><strong>{translated_variables['moderatelyRationalScore']}</strong>: {translated_variables['moderatelyRationalDescription']}</li>
                                <li><strong>{translated_variables['notRationalScore']}</strong>: {translated_variables['notRationalDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_rationality" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['influenceScale']}">
                        <HyperText name="influence_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyLikelyToInfluenceScore']}</strong>: {translated_variables['highlyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['moderatelyLikelyToInfluenceScore']}</strong>: {translated_variables['moderatelyLikelyToInfluenceDescription']}</li>
                                <li><strong>{translated_variables['notLikelyToInfluenceScore']}</strong>: {translated_variables['notLikelyToInfluenceDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_influence" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
            <View>
                <Collapse>
                    <Panel value="{translated_variables['alignmentScale']}">
                        <HyperText name="alignment_guidelines" size="4">
                            <ul>
                                <li><strong>{translated_variables['highlyAlignedScore']}</strong>: {translated_variables['highlyAlignedDescription']}</li>
                                <li><strong>{translated_variables['moderatelyAlignedScore']}</strong>: {translated_variables['moderatelyAlignedDescription']}</li>
                                <li><strong>{translated_variables['notAlignedScore']}</strong>: {translated_variables['notAlignedDescription']}</li>
                            </ul>
                        </HyperText>
                    </Panel>
                </Collapse>
                <View className="left-padded">
                    <Rating name="rating_alignment" toName="chat" maxRating="5" icon="star" size="medium" />
                </View>
            </View>
        </View>
    </View>
</View>
"""
        }
    }

    # Save each template as a separate JSON file
    for template_name, template_data in templates.items():
        language_value = get_language_value(lang)
        file_name = f"{output_folder_path}{lang}_{template_name} {language_value}.json"
        with open(file_name, 'w', encoding='utf-8') as f:
            json.dump(template_data, f, ensure_ascii=False, indent=4)

    print(f"Saved templates for {lang} to {file_name}")

# Step 2: Add projects to Label Studio

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
import os
import json
import requests
import shutil
import time

drive.mount('/content/drive', force_remount=True)

# Step 2: Set up paths
base_folder = '/content/drive/My Drive/BASE_FOLDER' # Replace with the actual file path to your folder. This should be the folder where the JSON files from the previous step are stored/
failed_folder = '/content/drive/My Drive/FAILED_FOLDER' # Replace with the actual file path to your folder. This should be the folder where any project that fails to upload to Label Studio is sent.
api_url = 'YOUR_URL' #Replace with the URL for your Label Studio instance's project API
api_token = 'YOUR_API_KEY' #Replace with your actual API key

# Set the maximum number of projects to send
max_projects_to_send = 5 # Change this to the desired number for testing. It's useful to leave this on 5 for testing as there are 5 main projects to be translated. When running the full script, set it to 800 or above.

# Ensure the failed uploads folder exists
if not os.path.exists(failed_folder):
    os.makedirs(failed_folder)

# Function to create a project
def create_project(project_data, file_path):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Token {api_token}'
    }
    response = requests.post(api_url, headers=headers, data=json.dumps(project_data))
    if response.status_code == 201:
        print(f"Project '{project_data['title']}' created successfully!")
    else:
        print(f"Failed to create project '{project_data['title']}'. Status code: {response.status_code}, Response: {response.text}")
        # Save the JSON file to the failed uploads folder
        shutil.copy(file_path, failed_folder)
        print(f"Saved failed project to {failed_folder}")
        # Print the label_config for debugging
        print("label_config:", project_data['label_config'])

# Step 3: Read JSON files and create projects
projects_sent = 0
for filename in os.listdir(base_folder):
    if filename.endswith('.json'):
        if projects_sent >= max_projects_to_send:
            break
        file_path = os.path.join(base_folder, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            project_data = json.load(file)

            # Construct the payload
            payload = {
                'title': project_data.get('title'),
                'description': project_data.get('description'),
                'label_config': project_data.get('label_config'),
                'expert_instruction': 'give feedback',
                'show_instruction': False,
                'show_skip_button': True,
                'enable_empty_annotation': False,
                'show_annotation_history': False,
                'reveal_preannotations_interactively': False,
                'show_collab_predictions': False,
                'maximum_annotations': 0,
                'color': '#FFFFFF',
                'control_weights': {}
            }

            # Create the project
            create_project(payload, file_path)
            projects_sent += 1

            # Add a 3-second delay between requests
            time.sleep(3)

print(f"{projects_sent} project(s) processed.")